In [19]:
import pandas as pd

In [ ]:
import os

In [29]:
import requests
import json

In [ ]:
df=pd.read_csv('data/'+os.listdir('data')[0])

In [ ]:
df.rename(columns={'#':'House Number'},inplace=True)

In [ ]:
df['json'] = df.apply(lambda x: x.to_json(), axis=1)

In [ ]:
df['Address']=df['House Number']+' '+df['Street Location']

In [ ]:
df['keyed_json']=df['Address']+": "+df['json']

In [ ]:
with open("data/weston.json", "w") as westondump:
    json.dump(df['keyed_json'].tolist(), westondump)

In [30]:
url='https://gist.githubusercontent.com/rselover/a82f17ec1a97538080940248880597fe/raw/69c797301be90b135eccc00d2015a28edc630b95/weston.json'

In [33]:
data = requests.get(url).json()

In [36]:
data[0].split(': ')[1]

'{"House Number":"1","Street Location":"ABERDEEN RD","ELA":"2,316","GLA (Above Grade)":"2,250","AYB":1933.0,"Acres":0.6,"FY24 Land Val":"$699,900","FY24 Bldg Val":"$534,400","FY24 Total Val":"$1,234,300"}'

In [37]:

# Load the JSON data from the URL
data = requests.get(url).json()

# Initialize an empty list to store the dictionaries
dicts = []

# Initialize a counter for the parcelid
parcelid = 1

# Iterate over the list of strings
for item in data:
    # Check if the item is a string
    if isinstance(item, str):
        # Split the string into the address and the property details
        address, details = item.split(': ', 1)
        
        # Convert the property details from a string to a dictionary
        details_dict = json.loads(details)
    else:
        # If the item is not a string, assign it a 'parcelid' and increment the parcelid
        address = f'parcelid_{parcelid}'
        details_dict = {}
        parcelid += 1
    
    # Combine the address and the property details into a single dictionary
    combined_dict = {'Address': address, **details_dict}
    
    # Append this dictionary to the list
    dicts.append(combined_dict)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dicts)

# Print the DataFrame
print(df)

             Address House Number Street Location     ELA GLA (Above Grade)  \
0      1 ABERDEEN RD            1     ABERDEEN RD   2,316             2,250   
1      4 ABERDEEN RD            4     ABERDEEN RD   1,900             1,700   
2      5 ABERDEEN RD            5     ABERDEEN RD   2,899             2,438   
3     10 ABERDEEN RD           10     ABERDEEN RD   1,442             1,253   
4     14 ABERDEEN RD           14     ABERDEEN RD   6,305             4,845   
...              ...          ...             ...     ...               ...   
4331     44 YOUNG RD           44        YOUNG RD  10,944             9,096   
4332     45 YOUNG RD           45        YOUNG RD   8,148             6,306   
4333     47 YOUNG RD           47        YOUNG RD   9,495             6,796   
4334     50 YOUNG RD           50        YOUNG RD   3,862             3,292   
4335      parcelid_1          NaN             NaN     NaN               NaN   

         AYB  Acres FY24 Land Val FY24 Bldg Val FY2

In [55]:
import numpy as np

In [60]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [62]:
df['GLA_Test']=df['GLA (Above Grade)'].str.replace(',','').astype(float)

In [63]:
df['$/AGSF']=df['Building_Value_Float']/df['GLA_Test']

KeyError: 'Building_Value_Float'

In [40]:
df['$/AGSF']=df['FY24 Total Val'].astype(float)/df['GLA (Above Grade)']

ValueError: could not convert string to float: '$1,234,300'

In [64]:
df.head()

,Address,House Number,Street Location,ELA,GLA (Above Grade),AYB,Acres,FY24 Land Val,FY24 Bldg Val,FY24 Total Val,GLA_Test
0,1 ABERDEEN RD,1,ABERDEEN RD,"2,316","2,250",1933.0,0.60,"$699,900","$534,400","$1,234,300",2250.0
1,4 ABERDEEN RD,4,ABERDEEN RD,"1,900","1,700",1932.0,0.41,"$604,800","$191,600","$796,400",1700.0
2,5 ABERDEEN RD,5,ABERDEEN RD,"2,899","2,438",1939.0,0.48,"$755,200","$271,500","$1,026,700",2438.0
3,10 ABERDEEN RD,10,ABERDEEN RD,"1,442","1,253",1937.0,0.55,"$610,800","$167,400","$778,200",1253.0
4,14 ABERDEEN RD,14,ABERDEEN RD,"6,305","4,845",2013.0,2.15,"$559,600","$1,963,600","$2,523,200",4845.0


In [65]:
dff=df[df['Address']=='144 SUDBURY RD']

In [70]:
dff['Acres'].values[0]

1.19